# **Como Funciona o Google Trends?**

O Google Trends é uma ferramenta disponibilizada gratuitamente para acompanhar e analisar a popularidade de palavras-chaves que foram buscadas na pesquisa do Google em uma janela de tempo.

**Como funciona o índice de popularidade?**

Baseado na período selecionado, na categoria, na região, no tipo de pesquisa e na quantidade de termos selecionados (01 à 05). O indíce é gerado com base na quantidade de pesquisas do Google para gerar um índice naquale período que vai de 0 à 100. Entendendo que:
- **00**: não há dados o suficiente para gerar uma nota.
- **50**: tem uma popularidade média.
- **100**: está no pico da popularidade.

<div style="text-align: center;">
  <img src="imagens\interest_over_time.png" alt="Interest by Over Time" width="500">
</div>

**Importando as Bibliotecas**

In [1]:
import pandas as pd
from pytrends.request import TrendReq
import time
from warnings import filterwarnings
filterwarnings('ignore')

# **Explorando as Funções do PyTrends**

Nessa seção há uma breve explicação das funções principais de extração de dados para o Google Trends.

## **Interest Over Time**

Essa função auxília na busca da popularidade de um termo em uma janela de tempo específica.

In [24]:
pytrend = TrendReq()

pytrend.build_payload(kw_list=['Elden Ring'],
                      timeframe='today 3-m', #Seleção do período, existem vários formatos possíveis
                      cat=0, #Seleção da categoria desejada, aqui tem o link para todas: https://github.com/pat310/google-trends-api/wiki/Google-Trends-Categories
                      geo='BR') #Seleção da região ou sub-região, como 'BR-SP'

'''
Existem também outros parâmetros como:
gprop - para filtrar o tipo de pesquisa, tipo: imagens, notícias, YouTube ou Google Shopping
tz - para selecionar um fuso-horário específico
'''

#Rodando a função que observa os dados ao decorrer do tempo
df = pytrend.interest_over_time()

Abaixo tem uma prévia dos dados que solicitamos:
- *Período:* 
Últimos 3 meses até hoje
- *Categoria:* 
Geral
- *Localidade:* 
Brasil

A solicitação retornou uma tabela trazendo a data, o valor do indicardor do Trends aquela data e se os dados são parciais ou não.

In [25]:
#Observando os dados do início da tabela
df.head()

,Elden Ring,isPartial
date,,
2024-07-21,100,False
2024-07-22,68,False
2024-07-23,65,False
2024-07-24,64,False
2024-07-25,58,False


In [26]:
#Observando os dados do final da tabela
df.tail()

,Elden Ring,isPartial
date,,
2024-10-17,14,False
2024-10-18,14,False
2024-10-19,21,False
2024-10-20,22,False
2024-10-21,16,True


## **Interest by Region**

Essa função retorna a popularidade de um termo em um país, macro-região, região ou cidade.

<div style="text-align: center;">
  <img src="imagens\interest_by_region.png" alt="Interest by Region" width="600">
</div>

In [82]:
pytrend = TrendReq()

pytrend.build_payload(kw_list=['Elden Ring'],
                      timeframe='today 1-m',
                      cat=0)

df = pytrend.interest_by_region(resolution='COUNTRY', 
                                inc_low_vol=True, 
                                inc_geo_code=True)

df.reset_index(inplace=True)

df.head()

,geoName,geoCode,Elden Ring
0,Afghanistan,AF,2
1,Albania,AL,21
2,Algeria,DZ,16
3,American Samoa,AS,0
4,Andorra,AD,6


In [78]:
pytrend = TrendReq()

pytrend.build_payload(kw_list=['Elden Ring'],
                      timeframe='today 1-m',
                      cat=0)

df = pytrend.interest_by_region(resolution='DMA', 
                                inc_low_vol=True, 
                                inc_geo_code=True)

df.reset_index(inplace=True)

df.head()

,geoName,geoCode,Elden Ring
0,Abilene-Sweetwater TX,662,62
1,Ain,FR-01,38
2,Aisne,FR-02,34
3,Albany GA,525,48
4,Albany-Schenectady-Troy NY,532,58


In [83]:
pytrend = TrendReq()

pytrend.build_payload(kw_list=['Elden Ring'],
                      timeframe='today 1-m',
                      cat=0)

df = pytrend.interest_by_region(resolution='CITY', 
                                inc_low_vol=True, 
                                inc_geo_code=True)

df.reset_index(inplace=True)

df.head()

,geoName,coordinates,Elden Ring
0,Aachen,"{'lat': 50.7753455, 'lng': 6.0838868}",61
1,Aarhus,"{'lat': 56.162939, 'lng': 10.203921}",0
2,Abington,"{'lat': 40.1198332, 'lng': -75.1253492}",0
3,Airdrie,"{'lat': 51.2917005, 'lng': -114.0143635}",0
4,Anchorage,"{'lat': 61.2180556, 'lng': -149.9002778}",62


## **Trending Searches**

Essa função retorna as principais trendings plataforma de acordo com o país.

<div style="text-align: center;">
  <img src="imagens\trending_now.png" alt="Trending Now" width="600">
</div>

In [76]:
pytrend = TrendReq()
df = pytrend.trending_searches(pn='brazil')
df

,0
0,Mega-Sena
1,Sport Recife x Botafogo-SP
2,Lula
3,Operário x Paysandu
4,Ponte Preta x Guarani
5,Jogo do corinthians hoje
6,Jogo do flamengo hoje
7,Roma x Inter
8,Libertadores
9,Resultado da mega-sena


# **-------------------------------------------------------------------------------------------------------------------**

# **Forma Simples de Coletar Vários Termos**

Abaixo segue um exemplo simples de código para extrair dados sobre pesquisas sobre jogos entre março e outubro de 2024.

**<span style="color:#ed6b72">IMPORTANTE!</span>** 

O Google Trends tem um limite máximo de 7 meses para pegar dados diários, após esse período, os dados se tornam semanais.

**Célula feita para ajustes de extração**

In [3]:
#Termos que vamos pequisar no Google Trends
termos = [
    'Elden Ring', 'Assassins Creed', 'Borderlands', 'Fallout', 'Dark Souls',
    'Castlevania', 'Fortnite', 'League of Legends', 'Valorant', 'Silent Hill'
]

#Escolha do Período - Padrão AAAA-MM-DD
data_incial = "2024-04-01"
data_final = "2024-10-17"

#Nome para o arquivo Excel
nome_do_excel = "df_termos_games.xlsx"

In [4]:
#Criando a variável com a data de pesquisa
periodo = data_incial + " " + data_final

#Construindo a tabela diária de termos
periodo_df = pd.date_range(start=data_incial, end=data_final, freq='D')
df_trends = pd.DataFrame(columns= termos,
                         index= periodo_df)

In [5]:
pytrend = TrendReq()

for i in termos:
    pytrend.build_payload(kw_list=[i], #Selecioando um termo por vez dentro da lista
                          timeframe=periodo) #Período de tempo para coleta
    
    #Chamando a função que extrai os dados do GT
    df = pytrend.interest_over_time() 
    
    #Deletando a coluna que indica se os dados que são parciais ou não
    df.drop(columns=['isPartil'], inplace=True, errors='ignore') 

    #Indicando o preenchimento na tabela construida na célula anterior
    if i in df.columns and not df[i].empty:
        datas_comuns = df.index.intersection(df_trends.index)

        df_trends.loc[datas_comuns, i] = df.loc[datas_comuns, i]

    #Tempo de Pausa entre cada ação em segundos
    time.sleep(15)

#Pode acontecer de alguns termos não terem tido relevância em um dia específico.
#Quando isso ocorre, pode acontecer de não retornar dados.
#Para evitar, preenchemos os locais vazios com zero, indicando baixa relevância.
df_trends.fillna(0, inplace=True)

#Exportação do arquivo para um excel
df_trends.to_excel(nome_do_excel)

df_trends.head()

,Elden Ring,Assassins Creed,Borderlands,Fallout,Dark Souls,Castlevania,Fortnite,League of Legends,Valorant,Silent Hill
2024-04-01,11,32,7,8,66,27,50,55,61,5
2024-04-02,8,26,6,7,58,23,49,50,56,5
2024-04-03,9,26,6,7,56,23,48,55,55,5
2024-04-04,8,27,6,7,55,23,45,51,57,5
2024-04-05,9,25,6,8,56,25,47,51,59,4
